# 설치 (colab 환경)

In [ ]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.0 MB/s 
     |████████████████████████████████| 16.7 MB 35.5 MB/s 
     |████████████████████████████████| 6.3 MB 57.2 MB/s 


In [ ]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 2.0 MB/s 


# Import

In [ ]:
# import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine
from haversine import haversine
from tqdm.notebook import tqdm
import numpy as np
import pymysql
import matplotlib.pyplot as plt

# DB Connect

In [ ]:
endpoint = 'dao.cdqd2bwz9q4s.ap-northeast-2.rds.amazonaws.com'
schema = 'dao'
db_connection_str = 'mysql+pymysql://admin:ekfkawnl@{}/{}'.format(endpoint, schema)
try :
    db_connection = create_engine(db_connection_str)
    conn = db_connection.connect()
except :
    print('fail to connect db')

# DataLoad

In [ ]:
def load_data_from_rds(tabel_name):
  sql = "SELECT * FROM {}".format(tabel_name)
  df = pd.read_sql(sql, db_connection)
  return df

# Preprocessing

In [ ]:
main_table = load_data_from_rds('생태통로_목록_0808')

In [ ]:
main_table

,번호,생태통로명,주소,도로명,도로번호,차선_수,위도_도,위도_분,위도_초,경도_도,경도_분,경도_초,위도_도분초,경도_도분초,위도_십진법,경도_십진법
0,1001,중동터널,서울특별시 구로구 궁동,계남큰길,None,왕복6차로,37,30.0,23.37000,126,49.0,51.56000,37 30 23.37,126 49 51.56,37.50649,126.83099
1,1002,개좌생태터널,부산광역시 금정구 회동동 산4-1,개좌로,0,왕복2차선,35,15.0,6.05000,129,8.0,27.31000,35 15 6.05,129 8 27.31,35.25168,129.14092
2,1004,곰내 생태통로,부산광역시 기장군 철마면 웅천리,군도,19호선,왕복2차선,35,18.0,20.80000,129,10.0,19.20000,35 18 20.8,129 10 19.2,35.30578,129.17200
3,1005,정관예림리 생태통로,부산광역시 기장군 정관면 예림리,국지도,60호,왕복4차선,35,19.0,46.80000,129,12.0,11.90000,35 19 46.8,129 12 11.9,35.32967,129.20331
4,1006,을숙도 생태통로,부산광역시 사하구 하단동,낙동남로,1240,왕복7차선,35,6.0,31.84632,128,56.0,28.46904,35 6 31.8463200000076,128 56 28.469039999984,35.10885,128.94124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,1564,삼척노곡,강원도 삼척시 원덕읍 노곡리 산146-3,국도,7호선,왕복4차선,37,12.0,33.19000,129,20.0,14.09000,37 12 33.19,129 20 14.09,37.20922,129.33725
532,1565,절토육교,경기도 의왕시 월암동 산46-3,서수원~의왕간 도로,지방도 309호선,왕복6차선,37,19.0,2.04000,126,57.0,59.00000,37 19 2.04,126 57 59.0,37.31723,126.96639
533,1566,동물이동통로,경기도 의왕시 고천동 산55-5,서수원~의왕간 도로,지방도309호선,왕복6차선,37,20.0,11.80000,126,58.0,23.42000,37 20 11.8,126 58 23.42,37.33661,126.97317
534,1567,None,경기도 영중면 거사리 산3-22번지 일원,중로,3-30,왕복2차선,37,59.0,30.60000,127,13.0,43.23000,37 59 30.6,127 13 43.23,37.99183,127.22868


In [ ]:
def calc_distance(bridge, factor, limit) :
  num_bridge = len(bridge)
  num_factor = len(factor)
  true_table = [[False for _ in range(num_factor)] for _ in range(num_bridge)]

  for b in tqdm(range(num_bridge)) :
    start = (bridge.iloc[b]['위도_십진법'], bridge.iloc[b]['경도_십진법'])
    for f in range(num_factor) :
      goal = (factor.iloc[f]['위도_십진법'], factor.iloc[f]['경도_십진법'])
      if haversine(start,goal,unit='km') <= limit :
        true_table[b][f] = True
  return np.array(true_table)

## 포유류

In [ ]:
ani = load_data_from_rds('포유류_전국자연환경조사_4차')
ani.head()

,objectid,학명,국명,위도,경도,조사일자,조사년도,시도명,layer,path
0,25228,"Apodemus agrarius (Pallas, 1771)",등줄쥐,126.477704,36.725967,2014-07-22,2014,충청남도,전국자연환경조사_4차_포유류_2014_전국,D:/다람쥐오래걷기/전국자연환경조사 4차 포유류/전국자연환경조사_4차_포유류_201...
1,25949,"Apodemus agrarius (Pallas, 1771)",등줄쥐,127.167039,37.941147,2014-07-22,2014,경기도,전국자연환경조사_4차_포유류_2014_전국,D:/다람쥐오래걷기/전국자연환경조사 4차 포유류/전국자연환경조사_4차_포유류_201...
2,16474,"Apodemus agrarius (Pallas, 1771)",등줄쥐,129.146806,36.940000,2014-09-16,2014,경상북도,전국자연환경조사_4차_포유류_2014_전국,D:/다람쥐오래걷기/전국자연환경조사 4차 포유류/전국자연환경조사_4차_포유류_201...
3,2037,"Apodemus agrarius (Pallas, 1771)",등줄쥐,128.870100,35.634910,2014-05-12,2014,경상북도,전국자연환경조사_4차_포유류_2014_전국,D:/다람쥐오래걷기/전국자연환경조사 4차 포유류/전국자연환경조사_4차_포유류_201...
4,30842,"Apodemus agrarius (Pallas, 1771)",등줄쥐,128.719364,36.808017,2014-08-04,2014,경상북도,전국자연환경조사_4차_포유류_2014_전국,D:/다람쥐오래걷기/전국자연환경조사 4차 포유류/전국자연환경조사_4차_포유류_201...


In [ ]:
ani['위도_십진법'] = ani['경도']
ani['경도_십진법'] = ani['위도']
ani.drop(['위도', '경도', 'path'], axis=1, inplace=True)
ani

,objectid,학명,국명,조사일자,조사년도,시도명,layer,위도_십진법,경도_십진법
0,25228,"Apodemus agrarius (Pallas, 1771)",등줄쥐,2014-07-22,2014,충청남도,전국자연환경조사_4차_포유류_2014_전국,36.725967,126.477704
1,25949,"Apodemus agrarius (Pallas, 1771)",등줄쥐,2014-07-22,2014,경기도,전국자연환경조사_4차_포유류_2014_전국,37.941147,127.167039
2,16474,"Apodemus agrarius (Pallas, 1771)",등줄쥐,2014-09-16,2014,경상북도,전국자연환경조사_4차_포유류_2014_전국,36.940000,129.146806
3,2037,"Apodemus agrarius (Pallas, 1771)",등줄쥐,2014-05-12,2014,경상북도,전국자연환경조사_4차_포유류_2014_전국,35.634910,128.870100
4,30842,"Apodemus agrarius (Pallas, 1771)",등줄쥐,2014-08-04,2014,경상북도,전국자연환경조사_4차_포유류_2014_전국,36.808017,128.719364
...,...,...,...,...,...,...,...,...,...
36259,8073,"Eutamias sibiricus (Laxmann), 1769",다람쥐,2018-05-09,2018,전라북도,전국자연환경조사_4차_포유류_2018_전국,35.603937,127.378426
36260,4010,"Eutamias sibiricus (Laxmann), 1769",다람쥐,2018-08-17,2018,강원도,전국자연환경조사_4차_포유류_2018_전국,37.396472,128.837444
36261,8076,"Eutamias sibiricus (Laxmann), 1769",다람쥐,2018-05-09,2018,전라북도,전국자연환경조사_4차_포유류_2018_전국,35.599633,127.384686
36262,8102,"Eutamias sibiricus (Laxmann), 1769",다람쥐,2018-05-09,2018,전라북도,전국자연환경조사_4차_포유류_2018_전국,35.615764,127.447300


In [ ]:
target_ani = ['멧돼지', '고라니', '노루', '너구리', '오소리', '족제비','멧토끼', '청설모', '다람쥐']

In [ ]:
target_animals = ani[ani['국명'].isin(target_ani)]
target_animals

,objectid,학명,국명,조사일자,조사년도,시도명,layer,위도_십진법,경도_십진법
378,39948,"Sciurus vulgaris Linnaeus, 1758",청설모,2014-09-23,2014,충청남도,전국자연환경조사_4차_포유류_2014_전국,36.713611,126.225556
379,39952,"Sciurus vulgaris Linnaeus, 1758",청설모,2014-07-20,2014,충청남도,전국자연환경조사_4차_포유류_2014_전국,36.719444,126.130833
380,42882,"Sciurus vulgaris Linnaeus, 1758",청설모,2014-06-28,2014,경상북도,전국자연환경조사_4차_포유류_2014_전국,35.600150,128.641530
381,34744,"Sciurus vulgaris Linnaeus, 1758",청설모,2014-10-11,2014,경상북도,전국자연환경조사_4차_포유류_2014_전국,36.788417,129.103611
382,39957,"Sciurus vulgaris Linnaeus, 1758",청설모,2014-07-20,2014,충청남도,전국자연환경조사_4차_포유류_2014_전국,36.720833,126.133333
...,...,...,...,...,...,...,...,...,...
36259,8073,"Eutamias sibiricus (Laxmann), 1769",다람쥐,2018-05-09,2018,전라북도,전국자연환경조사_4차_포유류_2018_전국,35.603937,127.378426
36260,4010,"Eutamias sibiricus (Laxmann), 1769",다람쥐,2018-08-17,2018,강원도,전국자연환경조사_4차_포유류_2018_전국,37.396472,128.837444
36261,8076,"Eutamias sibiricus (Laxmann), 1769",다람쥐,2018-05-09,2018,전라북도,전국자연환경조사_4차_포유류_2018_전국,35.599633,127.384686
36262,8102,"Eutamias sibiricus (Laxmann), 1769",다람쥐,2018-05-09,2018,전라북도,전국자연환경조사_4차_포유류_2018_전국,35.615764,127.447300


In [ ]:
%%time
true_table = calc_distance(main_table, target_animals, 5)

  0%|          | 0/536 [00:00<?, ?it/s]

CPU times: user 1h 7min 36s, sys: 8.02 s, total: 1h 7min 44s
Wall time: 1h 8min 4s


In [ ]:
print(true_table.shape)
print(true_table.sum())

(536, 29259)
13579


In [ ]:
df = pd.DataFrame(columns=['번호', '종명'])
df

,번호,종명


In [ ]:
## true table
for b in tqdm(range(true_table.shape[0])) :
  for f in range(true_table.shape[1]) :
    if true_table[b][f] :
      temp = pd.DataFrame([[main_table.iloc[b]['번호'], 
                            target_animals.iloc[f]['국명']]],
                          columns=df.columns)
      df = pd.concat([df, temp])
df

  0%|          | 0/536 [00:00<?, ?it/s]

,번호,종명
0,1001,고라니
0,1001,너구리
0,1002,멧토끼
0,1002,다람쥐
0,1002,청설모
...,...,...
0,1568,고라니
0,1568,멧돼지
0,1568,너구리
0,1568,너구리


In [ ]:
# 수행결과 저장
df.to_csv('/content/drive/MyDrive/Colab Notebooks/생태통로_주변_동물분포.csv', index=False)

In [ ]:
# 주변에서 동물종이 발견된 생태통로 개수
df.groupby('번호').count().__len__()

441

In [ ]:
grouped_df = df.groupby('번호')

In [ ]:
result= pd.DataFrame(main_table['번호'])

### 생태통로 주변에서 발견된 포유류 종의 개수

In [ ]:
unique_ = pd.DataFrame(grouped_df['종명'].nunique())
unique_.rename(columns={'종명' : '주변 동물종 개수'}, inplace=True)
unique_

,주변 동물종 개수
번호,
1001,2
1002,4
1004,4
1005,4
1006,3
...,...
1564,6
1565,1
1566,3


### 생태통로 주변에서 발견된 포유류 빈도수

In [ ]:
cnt = pd.DataFrame(grouped_df['종명'].count())
cnt.rename(columns={'종명':'주변동물 출현빈도'}, inplace=True)
cnt

,주변동물 출현빈도
번호,
1001,2
1002,14
1004,16
1005,8
1006,10
...,...
1564,17
1565,2
1566,7


### 빈도와 다양도 join

In [ ]:
result = result.merge(unique_, how='outer', on='번호')
result = result.merge(cnt, how='outer', on='번호')
result

,번호,주변 동물종 개수,주변동물 출현빈도
0,1001,2.0,2.0
1,1002,4.0,14.0
2,1004,4.0,16.0
3,1005,4.0,8.0
4,1006,3.0,10.0
...,...,...,...
531,1564,6.0,17.0
532,1565,1.0,2.0
533,1566,3.0,7.0
534,1567,6.0,9.0


In [ ]:
result.fillna(0, inplace=True)
result = result.astype({'주변 동물종 개수': 'int',
                        '주변동물 출현빈도': 'int'})

In [ ]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536 entries, 0 to 535
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   번호         536 non-null    int64
 1   주변 동물종 개수  536 non-null    int64
 2   주변동물 출현빈도  536 non-null    int64
dtypes: int64(3)
memory usage: 16.8 KB


In [ ]:
# 결과 확인
result

,번호,주변 동물종 개수,주변동물 출현빈도
0,1001,2,2
1,1002,4,14
2,1004,4,16
3,1005,4,8
4,1006,3,10
...,...,...,...
531,1564,6,17
532,1565,1,2
533,1566,3,7
534,1567,6,9


### MainData와 join

In [ ]:
result.to_sql('re_MainData_inner_HK', db_connection, if_exists='replace', index=False)